In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.839445 auc:0.4291
epoch:  20 loss:0.395238 auc:0.9011
epoch:  40 loss:0.372818 auc:0.9069
epoch:  60 loss:0.368106 auc:0.9075
epoch:  80 loss:0.366758 auc:0.9079
epoch: 100 loss:0.365828 auc:0.9082
epoch: 120 loss:0.364723 auc:0.9087
epoch: 140 loss:0.362825 auc:0.9095
epoch: 160 loss:0.358915 auc:0.9110
epoch: 180 loss:0.353265 auc:0.9124
epoch: 200 loss:0.347435 auc:0.9137
epoch: 220 loss:0.341596 auc:0.9153
epoch: 240 loss:0.331860 auc:0.9178
epoch: 260 loss:0.322501 auc:0.9202
epoch: 280 loss:0.313230 auc:0.9225
epoch: 300 loss:0.303815 auc:0.9252
epoch: 320 loss:0.298937 auc:0.9272
epoch: 340 loss:0.297077 auc:0.9265
epoch: 360 loss:0.286665 auc:0.9300
epoch: 380 loss:0.281384 auc:0.9312
epoch: 400 loss:0.278066 auc:0.9318
epoch: 420 loss:0.275145 auc:0.9325
epoch: 440 loss:0.271845 auc:0.9334
epoch: 460 loss:0.270849 auc:0.9336
epoch: 480 loss:0.267452 auc:0.9346
epoch: 500 loss:0.266455 auc:0.9348
epoch: 520 loss:0.264624 auc:0.9353
epoch: 540 loss:0.263103 auc

In [6]:
true_datas.reset_index(drop=True).to_csv("true_ctrp.csv")
predict_datas.reset_index(drop=True).to_csv("pred_ctrp.csv")